1. Load the model Yeast8

In [43]:
from cobra.io import read_sbml_model
from cobra import Reaction, Metabolite

model = read_sbml_model('yeastGEM.xml')

2. Running model with standard Meidum 

In [45]:
# optimize for biomass growth with standard medium

In [47]:
model.optimize()                               
print(model.objective.value)

0.08374778664999834


In [49]:
#Create reaction dictionary

In [51]:
xyodict = {1: "r_2105", # xylitol transport
           2: "r_1719", # D-xylose transport
           3: "r_1093", # xylose reductase
           4: "r_1092"  # xylitol dehydrogenase
          }

In [53]:
#Check the standard medium. (medium that has as carbon source Glucose)


In [55]:
medium_glu = model.medium
for element in medium_glu.keys():
    print(model.reactions.get_by_id(element).name, model.reactions.get_by_id(element), medium_glu[element])

ammonium exchange r_1654: s_0420[e] <=>  1000.0
D-glucose exchange r_1714: s_0565[e] <=>  1.0
H+ exchange r_1832: s_0796[e] <=>  1000.0
iron(2+) exchange r_1861: s_0925[e] <=>  1000.0
oxygen exchange r_1992: s_1277[e] <=>  1000.0
phosphate exchange r_2005: s_1324[e] <=>  1000.0
potassium exchange r_2020: s_1374[e] <=>  1000.0
sodium exchange r_2049: s_1438[e] <=>  1000.0
sulphate exchange r_2060: s_1468[e] <=>  1000.0
water exchange r_2100: s_0805[e] <=>  1000.0
chloride exchange r_4593: s_4200[e] <=>  1000.0
Cu2(+) exchange r_4594: s_4201[e] <=>  1000.0
Mn(2+) exchange r_4595: s_4202[e] <=>  1000.0
Zn(2+) exchange r_4596: s_4203[e] <=>  1000.0
Mg(2+) exchange r_4597: s_4204[e] <=>  1000.0
Ca(2+) exchange r_4600: s_4199[e] <=>  1000.0


In [57]:
#Run the model with the standard medium 

In [ ]:
solution = model.optimize()
print(model.summary())
for key in xyodict.keys():
    print(model.reactions.get_by_id(xyodict[key]).name, model.reactions.get_by_id(xyodict[key]).flux)

We can see how when the model is run with the standard medium (no xylose is present) the Xylitol transporter, Xylose transporter, Xylose Reductase (XR) and Xylitol Dehydrogenase (XDH) have flux equal to 0.  That means that with out Xylose in the media all the pathway is not active.

3. Medium Change, trying different proportions of Glucose and Xylose

3.1. Medium without glucose and with Xylose

In [ ]:

medium_xyl = medium_glu
with model: 
    del medium_xyl['r_1714']
    medium_xyl['r_1718'] = 1.0

for element in medium_xyl.keys():
    print(model.reactions.get_by_id(element).name, model.reactions.get_by_id(element), medium_xyl[element])

In [ ]:
with model:
    model.medium = medium_xyl
    solution2 = model.optimize()
    print(model.summary())
    for key in xyodict.keys():
        print(model.reactions.get_by_id(xyodict[key]).name, model.reactions.get_by_id(xyodict[key]).flux)

The growth decreases to 0.067 when the growth media insted of having glucose just have xylose. Now it can also be seen how the  the Xylose transport, the XR and the XDH are active, however there is not Xylitol transport. To better undertand the metabolism of cervisiae with th emodification that have to be applied, it is need to do the following modifications: 
    - Overexxpress the intake of Xylose, through overexpressing the reaction r_
    - Over express the enzyme Xylose Reducatse which coverts the Xylose in to Xylitol
    - Knock out the gene encoding for the enzyme Xylitol Dehydrogenase which trnasforms Xylitol to Xyluslose, losing yield of our product.
    - Overexpresse the transport of Xylitol outside of the cell
    

In [ ]:
with model:
    del medium_xyl['r_1714']
    model.medium = medium_xyl # no glucose and xylose present in the medium
       
   
    model.reactions.r_1719.bounds = (1,10) # regulation for xylose transporter
    model.reactions.r_1093.bounds = (1,10) # regulation for xylose reductase
    model.reactions.r_2104.bounds = (0,2) # regulation for xylitol transporter
    model.genes.YLR070C.knock_out() #knnockout for Xylitol Dehydrogenase


    
    ## solve model
    # model.objective = model.reactions.r_2104 # objective is maximum xylitol production
    solution3 = model.optimize()
    print(model.summary())
    for key in xyodict.keys():
        print(model.reactions.get_by_id(xyodict[key]).name, model.reactions.get_by_id(xyodict[key]).flux)
    print(model.reactions.r_2111.name, model.reactions.r_2111.flux)
    print(model.reactions.r_1714.name, model.reactions.r_1714.flux)
    print(model.reactions.r_1166.name, model.reactions.r_1166.flux)